 Before we do that though, lets think about **why** we might want an ROI.

### What is an ROI good for?

In the case of the planar ROI, what we have essentially done is defined a coordinate in a single dimension that is of particular interest to is.  For example, if we created a planar ROI at 10 Y in [ACPC space](https://bids-specification.readthedocs.io/en/latest/99-appendices/08-coordinate-systems.html#ieeg-specific-coordinate-systems) we would obtain a large number of points.  Specifically the X and Z coordinates for these points would correspond to all possible combinations of X and Z values between the minimum and maximum dimensions of the associated source image (i.e. a T1 or DWI image), sampled at some regular interval (i.e. 1 mm).  The Y coordinates, on the other hand, would all be 10.  So how can we use this?

#### Asking a question with an ROI

Now that we have our ROI we are now capable of asking (in a mathematical sense) a very specific question:  **which of our streamlines cross (i.e. "intersect with") this plane?**  We do this by determining, for each streamline, if **any** coordinate from the streamline is within some distance threshold (i.e. .5 mm) of any coordinate of the ROI.  In practice, this ends up being moderately computationally expensive.  Why?  Because, for each coordinate of the streamline, our intersection algorithm has to compute the distance between that coordinate, and each coordinate of the ROI (which equals [number of streamline nodes] * [number of ROI coordinates] computations).  As a consequence of this, the smaller the step size between the nodes in a streamline or the more densely sampled the ROI is, the longer this operation takes.  Once we have applied this opertion, we can then select those streamlines which meet the specified criteria.

> **A VERY IMPORTANT CAVIOT**:  A careful and attentive reader of the preceeding paragraph will note that it wasn't entirely true.  Specifically, if you translate the mathematical operation it described, you'll realize that what we really asked was "**Which of our streamlines have nodes that are within [some distance] of a coordinate of this ROI?**".  There are certian circumstances where the operation would return a true (i.e. "this streamline meets the specified criteria") for a given streamline even though the streamline didn't actually "intersect" the ROI.  How?  Imagine that we had a planar ROI and a streamline that traveled perfectly perpendicular with it, such that it got to within the threshold distance of the planar ROI _but never actually crossed it_.  Indeed, the streamline doesn't even need to travel alongside the planar ROI the entire time--it could just move to within the threshold distance, and then move away again (so long as at least one node meets this criteria).  In either case the algorithm we just described would still return a True for that streamline.  But this isn't the only problem.

> In addition to the aforementioned possibility, it's also possible that this algorithm (which is, in essence, the one that is typically used in segmentation) might **fail to detect** a streamlinehet that we would consider to be intersecting with the ROI.  How?  If we think back to our previous discussion of step-sizes between streamline nodes, we realize that its possible for there to actually be fairly sizable amounts of distance between nodes, particularly if the streamlines have been ["compressed"](https://dipy.org/documentation/1.0.0./examples_built/streamline_length/).  If the inter-node distance is enough to where the streamline "traverses" the ROI (i.e. there are nodes on either side of the ROI), but no individual node is within the distance threshold, then the algorithm will fail to detect this intersection.  For example, if the nearest nodes on either side of a planar ROI are 2 mm from the plane, but the threshold distance is .5, the "intersection" will go unnoticed.  This is particularly an issue with planar ROIs, but less so with fuller, volumetric ROIS (e.g. spheres).  

For the moment, there's really nothing we can do about these possibilities.  In the first case, we may not be too worried about it, since .5 is pretty close, and we'd probably be comfortable calling that an intersection.  In the latter case, we're probably not comfortable with that outcome, but we can safeguard against it by ensuring that our streamlines are sampled at a rate that takes into account the threshold distance of our intersection algorithm.  Regardless, its time to make and examine our first planar ROI.